In [1]:
import os
from river import datasets
from confluent_kafka import Producer,Consumer
import certifi
import time
import json

In [2]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [3]:
topic = 'malicious_url_events'

In [4]:
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,
            'batch.num.messages': 2048,
            #'queue.buffering.max.messages': 100,
            'linger.ms': 100,
            'client.id': 'producer-icde-2023'}
producer = Producer(conf)    

In [5]:
max_size=10000
dataset = datasets.MaliciousURL()
data = dataset.take(max_size)


In [7]:
end=0
cnt = 0
st = time.time()
abs_st = time.time()
for f, y in data:
    cnt = cnt + 1    
    d = {}
    d['f']=f
    d['y']=str(y).lower()
    d['st']=time.time()  
            
    v= json.dumps(d).encode('utf-8')
    try:
        producer.produce(topic, value=v, key=str(cnt))
    except:
      print(f'Queue full, flushing {cnt}')
      producer.flush()
      producer.produce(topic, value=v, key=str(cnt))
    if cnt%1024==0:           
        #end = time.time()
        print(f'flushing count - {cnt}, time taken in seconds- {end-st} ')        
        producer.flush()
        time.sleep(1)
        st = time.time()        
        
producer.flush()
end = time.time()
print(f'final flushing count - {cnt}, time taken in seconds- {end-abs_st} ')        

flushing count - 1024, time taken in seconds- -1679402607.623565 
flushing count - 2048, time taken in seconds- -1679402608.959284 
flushing count - 3072, time taken in seconds- -1679402610.2956533 
flushing count - 4096, time taken in seconds- -1679402611.6653483 
flushing count - 5120, time taken in seconds- -1679402613.0497224 
flushing count - 6144, time taken in seconds- -1679402614.4050431 
flushing count - 7168, time taken in seconds- -1679402615.796914 
flushing count - 8192, time taken in seconds- -1679402617.153735 
final flushing count - 8976, time taken in seconds- 11.249021530151367 


In [18]:
os.environ['PREDICTION_TOPIC_PREFIX']='malicious_url_predictions'
os.environ['PREDICTION_TOPIC_SUFFIX']='_16'
prediction_topic_prefix = os.environ['PREDICTION_TOPIC_PREFIX']
prediction_topic_suffix=os.environ['PREDICTION_TOPIC_SUFFIX']
PREDICTION_TOPIC=f'{prediction_topic_prefix}{prediction_topic_suffix}'

In [21]:
from river import metrics
auc = metrics.ROCAUC()
f1 = metrics.F1()
recall = metrics.MicroRecall()
predictions_consumer_conf = {'bootstrap.servers': bsts,
                     'sasl.username': user,
                     'sasl.password': password,
                     'sasl.mechanism': 'PLAIN',
                     'security.protocol': 'SASL_SSL',
                     'ssl.ca.location': certifi.where(),
                     'group.id': 'prediction-grp',
                     'enable.auto.commit': True,
                     'auto.offset.reset': 'earliest'}
predictions_consumer = Consumer(predictions_consumer_conf)    
predictions_consumer.subscribe([PREDICTION_TOPIC])

In [22]:
{'bootstrap.servers': bsts,
                     'sasl.username': user,
                     'sasl.password': password,
                     'sasl.mechanism': 'PLAIN',
                     'security.protocol': 'SASL_SSL',
                     'ssl.ca.location': certifi.where(),
                     'group.id': 'prediction-grp',
                     'enable.auto.commit': True,
                     'auto.offset.reset': 'latest'}

{'bootstrap.servers': 'pkc-6ojv2.us-west4.gcp.confluent.cloud:9092',
 'sasl.username': '3LZS4LPMJGHPXAOE',
 'sasl.password': '8sekIsdlanX6wuK+FQhZwHFvxVD/mQhfHS3Vi+pi7JqAOttktzucH2KZRy+Ip9gI',
 'sasl.mechanism': 'PLAIN',
 'security.protocol': 'SASL_SSL',
 'ssl.ca.location': '/opt/conda/lib/python3.9/site-packages/certifi/cacert.pem',
 'group.id': 'prediction-grp',
 'enable.auto.commit': True,
 'auto.offset.reset': 'latest'}

In [23]:

durs = []
mem_usage = []
cnt = 0
while(cnt<10000):
    msg = predictions_consumer.poll(timeout=0.1)
    
    if msg is None: continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                         (msg.topic(), msg.partition(), msg.offset()))
    else:            
        message = json.loads(msg.value().decode("utf-8"))
        if(cnt%1000==0):
            print(message)
            
        cnt = cnt + 1
        score = message['score']
        y = message['y']
        durs.append(message['duration'])
        mem_usage.append(message['mem_usage'])
        auc = auc.update(y, score)
        f1 = f1.update(y, score)
        recall = recall.update(y, score)
            

{'y': False, 'score': None, 'duration': 1.2357680797576904, 'learn_ds': 0.006380319595336914, 'mem_usage': 530772}


TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
import statistics
mean = statistics.mean(durs)
median = statistics.median(durs)
max_dur = max(durs)
min_dur = min(durs)
avg_dur = min(durs)
total_records = len(durs)
memory_usage = statistics.mean(mem_usage)
print(max_dur)
print(min_dur)
print(avg_dur)
print(auc)
print(f1)
print(recall)
print(memory_usage)

In [ ]:
from river import datasets
from river import metrics
from river import tree
from river import ensemble
from river import evaluate
from river import compose
from river import naive_bayes
from time import time

from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing

In [ ]:
model = ensemble.AdaptiveRandomForestClassifier(seed=8, leaf_prediction="mc")

In [ ]:
dir(ensemble)